In [33]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

import requests
from dotenv import load_dotenv
import os
load_dotenv()

True

In [34]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [35]:
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [36]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [37]:
infos = list(set(extract_text_items(data)))

In [38]:
template = """
Fait un résumé concis des informations suivantes : {informations}
"""

In [39]:
prompt = PromptTemplate.from_template(template)


In [40]:
chain = (
    {'informations':RunnablePassthrough()} 
    | prompt 
    | gpt_turbo
)

In [41]:
infos

["Faits nouveaux : passage en vigilance orange vent violents sur le Finistère pour un début d'évènement à 16h ce lundi. Situation générale : Une dépression nommée PIERRICK se présente en entrée de Manche, proche du nord Finistère en ce début d'après-midi de lundi puis circule le long des côtes sud de l'Angleterre, engendrant des vents violents et un risque de vagues-submersion.",
 'Une incertitude persiste sur le positionnement exacte du creusement lié à PIERRICK. Ainsi on privilégie un scénario avec de violentes rafales (autour de 110 km/h dans les terres) principalement sur un quart nord-Ouest du Finistère et proche du littoral, mais un positionnement légèrement plus sud pourrait apporter également des vents d\'une violence similaire sur le sud du département. Une aggravation de la vigilance "vagues-submersion" pour les départements de l\'est de la Manche n\'est pas exclue pour la marée haute de mardi après-midi.',
 'Conjonction de grandes marées et de fortes vagues nécessitant une v

In [42]:
answer = chain.invoke(infos)

In [43]:
print(answer)


La dépression PIERRICK se déplace vers l'est et provoque des vents violents et de fortes vagues sur la côte ouest de la France. Une vigilance orange est en place pour le Finistère et une vigilance jaune pour les autres départements de la façade atlantique. Les niveaux d'eau sont élevés en raison des grandes marées et peuvent causer des submersions marines. La dépression continuera à se déplacer vers l'est et affectera également les départements de la Manche et du Calvados. Des rafales de vent jusqu'à 130 km/h sont prévues sur les îles et la côte du Finistère. 
